# Xgboost implementation

In [502]:
import pandas as pd
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit, HalvingGridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import numpy as np
import matplotlib.pyplot as plt

In [503]:
df = pd.read_csv('../clean_data_2.csv')

In [504]:
df.head(5)

,lag_public_perc_che_2,lag_public_perc_che_1,lag_price_unit_2,lag_price_unit_1,lag_price_month_2,lag_price_month_1,lag_insurance_perc_che_2,lag_insurance_perc_che_1,lag_che_perc_gdp_2,lag_che_perc_gdp_1,...,ind_age,che_perc_gdp,launch_date,date,insurance_perc_che,population,price_month,price_unit,public_perc_che,target
0,1.835821,1.835821,1.013784,1.013784,1.006444,1.006444,1.893333,1.893333,1.665879,1.665879,...,0,1.665879,2014-06-01,2014-06-01,1.893333,1.008039,1.006444,1.013784,1.835821,1.000784
1,1.835821,1.835821,1.013784,1.013784,1.006444,1.006444,1.893333,1.893333,1.665879,1.665879,...,3,1.689515,2014-06-01,2014-06-01,1.495682,1.023562,1.121036,1.626677,1.779325,1.000000
2,1.835821,1.779325,1.013784,1.626677,1.006444,1.121036,1.893333,1.495682,1.665879,1.689515,...,0,1.665879,2014-06-01,2014-06-01,1.893333,1.008039,1.121036,3.144874,1.835821,1.002258
3,1.779325,1.835821,1.626677,3.144874,1.121036,1.121036,1.495682,1.893333,1.689515,1.665879,...,0,2.051770,2014-06-01,2014-06-01,1.000000,1.253186,1.121036,1.213446,1.805970,1.068761
4,1.835821,1.805970,3.144874,1.213446,1.121036,1.121036,1.893333,1.000000,1.665879,2.051770,...,0,2.059130,2014-06-01,2014-06-01,2.013333,1.639352,1.018589,1.008708,1.880597,1.036312


In [505]:
# df = df.drop(['cluster_nl'], axis=1)

In [506]:
cutoff_date = pd.to_datetime('2021-01-01')
df['date'] = pd.to_datetime(df['date'])
df['launch_date'] = pd.to_datetime(df['launch_date'])
df['age'] = (df['date'].dt.year - df['launch_date'].dt.year) * 12 + (df['date'].dt.month - df['launch_date'].dt.month)

# df['month'] = df['date'].dt.month
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
# cutoff_date = pd.to_datetime('2022-12-01')
train_df = df[df['date'] < cutoff_date]
test_df = df[df['date'] >= cutoff_date]

In [507]:
test_df

,age,lag_public_perc_che_2,lag_public_perc_che_1,lag_price_unit_2,lag_price_unit_1,lag_price_month_2,lag_price_month_1,lag_insurance_perc_che_2,lag_insurance_perc_che_1,lag_che_perc_gdp_2,...,ind_age,che_perc_gdp,launch_date,date,insurance_perc_che,population,price_month,price_unit,public_perc_che,target
82666,17,2.000000,1.880597,1.569604,2.495495,1.453418,1.296481,2.000000,1.106667,1.977701,...,3,1.264991,2019-08-01,2021-01-01,1.266667,1.040040,2.326372,2.598921,1.686567,1.013466
82667,4,1.880597,1.686567,2.495495,2.598921,1.296481,2.326372,1.106667,1.266667,1.753874,...,0,1.713685,2020-09-01,2021-01-01,1.440000,1.050181,1.002319,1.012425,1.552239,1.022224
82668,28,1.686567,1.552239,2.598921,1.012425,2.326372,1.002319,1.266667,1.440000,1.264991,...,27,1.910453,2018-09-01,2021-01-01,1.000000,1.175666,1.917042,2.450769,1.865672,2.971784
82669,59,1.552239,1.865672,1.012425,2.450769,1.002319,1.917042,1.440000,1.000000,1.713685,...,0,1.932831,2016-02-01,2021-01-01,1.495682,1.048995,1.384951,1.783161,2.029851,1.057714
82670,6,1.865672,2.029851,2.450769,1.783161,1.917042,1.384951,1.000000,1.495682,1.910453,...,0,1.689515,2020-07-01,2021-01-01,1.920000,1.051810,1.019102,1.018917,2.044776,1.140758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118912,25,1.552239,1.843706,1.001084,1.008337,1.121036,1.005084,1.440000,1.040000,1.684065,...,0,2.058055,2020-11-01,2022-12-01,1.495682,1.049808,1.054007,1.100336,2.029851,1.203657
118913,41,1.843706,2.029851,1.008337,1.100336,1.005084,1.054007,1.040000,1.495682,1.918488,...,0,1.819485,2019-07-01,2022-12-01,1.173333,1.008985,1.008317,1.029630,1.955224,1.109272
118914,51,2.029851,1.955224,1.100336,1.029630,1.054007,1.008317,1.495682,1.173333,2.058055,...,0,1.491552,2018-09-01,2022-12-01,1.826667,1.121505,1.017259,1.018310,1.926795,1.343341
118915,27,1.955224,1.926795,1.029630,1.018310,1.008317,1.017259,1.173333,1.826667,1.819485,...,2,2.020277,2020-09-01,2022-12-01,1.495682,1.520144,1.960978,2.490911,1.985847,1.266831


In [508]:
## Drop date launch date
train_df = train_df.drop(['date', 'launch_date'], axis=1)
test_df = test_df.drop(['date', 'launch_date'], axis=1)

In [509]:
test_df

,age,lag_public_perc_che_2,lag_public_perc_che_1,lag_price_unit_2,lag_price_unit_1,lag_price_month_2,lag_price_month_1,lag_insurance_perc_che_2,lag_insurance_perc_che_1,lag_che_perc_gdp_2,...,brand_encode,area_encode,ind_age,che_perc_gdp,insurance_perc_che,population,price_month,price_unit,public_perc_che,target
82666,17,2.000000,1.880597,1.569604,2.495495,1.453418,1.296481,2.000000,1.106667,1.977701,...,0.548255,0.408879,3,1.264991,1.266667,1.040040,2.326372,2.598921,1.686567,1.013466
82667,4,1.880597,1.686567,2.495495,2.598921,1.296481,2.326372,1.106667,1.266667,1.753874,...,0.000487,0.649907,0,1.713685,1.440000,1.050181,1.002319,1.012425,1.552239,1.022224
82668,28,1.686567,1.552239,2.598921,1.012425,2.326372,1.002319,1.266667,1.440000,1.264991,...,0.159736,0.408879,27,1.910453,1.000000,1.175666,1.917042,2.450769,1.865672,2.971784
82669,59,1.552239,1.865672,1.012425,2.450769,1.002319,1.917042,1.440000,1.000000,1.713685,...,0.359678,0.408879,0,1.932831,1.495682,1.048995,1.384951,1.783161,2.029851,1.057714
82670,6,1.865672,2.029851,2.450769,1.783161,1.917042,1.384951,1.000000,1.495682,1.910453,...,0.492476,0.709365,0,1.689515,1.920000,1.051810,1.019102,1.018917,2.044776,1.140758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118912,25,1.552239,1.843706,1.001084,1.008337,1.121036,1.005084,1.440000,1.040000,1.684065,...,0.368762,0.319773,0,2.058055,1.495682,1.049808,1.054007,1.100336,2.029851,1.203657
118913,41,1.843706,2.029851,1.008337,1.100336,1.005084,1.054007,1.040000,1.495682,1.918488,...,0.960980,0.877121,0,1.819485,1.173333,1.008985,1.008317,1.029630,1.955224,1.109272
118914,51,2.029851,1.955224,1.100336,1.029630,1.054007,1.008317,1.495682,1.173333,2.058055,...,0.240633,0.642469,0,1.491552,1.826667,1.121505,1.017259,1.018310,1.926795,1.343341
118915,27,1.955224,1.926795,1.029630,1.018310,1.008317,1.017259,1.173333,1.826667,1.819485,...,0.961038,0.408879,2,2.020277,1.495682,1.520144,1.960978,2.490911,1.985847,1.266831


In [510]:
train_x = train_df.iloc[:,:-1]
train_y = train_df.iloc[:, -1]

test_x = test_df.iloc[:, :-1]
test_y = test_df.iloc[:, -1]

In [511]:
test_y

82666     1.013466
82667     1.022224
82668     2.971784
82669     1.057714
82670     1.140758
            ...   
118912    1.203657
118913    1.109272
118914    1.343341
118915    1.266831
118916    1.001763
Name: target, Length: 36251, dtype: float64

In [512]:
n_split = 5
tscv = TimeSeriesSplit(n_splits=n_split)

In [513]:
parameters = { 'objective':['reg:pseudohubererror', 'reg:squarederror'],
              'learning_rate': [.01, 0.05, .1], #so called `eta` value
              'max_depth': [3, 5, 6, 7],
              'min_child_weight': [1, 3, 5],
              'subsample': [0.8, 1.0],
              'colsample_bytree': [0.8, 1.0],
              'n_estimators': [500]}

In [514]:
modelcv = XGBRegressor()

In [515]:
model_grid = HalvingGridSearchCV(
    modelcv, 
    parameters, 
    cv = tscv, 
    scoring = 'r2', 
    n_jobs=-1,
    return_train_score=True)

In [516]:
model_grid.fit(train_x, train_y)

d:\personalProject\NovartisDatathon\.venv\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


HalvingGridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                    estimator=XGBRegressor(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=N...
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                    n_jobs=-1,
                    param_grid={'colsample_bytree': [0.8, 1.0],
                                'learning_rate': [0.01, 0.05, 0.1],
                                'max_depth': [3, 5, 6, 7],
                                'min_child_weight': [1, 3, 5],
                                'n_estimators': [500],
                                'objective': ['reg:pseudohubererror',
                                              'reg:squarederror'],
                                'subsample': [0.8, 1.0]},
                    scoring='r2')

In [517]:
model_grid.best_params_

{'colsample_bytree': 1.0,
 'learning_rate': 0.05,
 'max_depth': 7,
 'min_child_weight': 5,
 'n_estimators': 500,
 'objective': 'reg:pseudohubererror',
 'subsample': 0.8}

In [518]:
best_model = model_grid.best_estimator_

In [519]:
pred = best_model.predict(test_x)

In [520]:
mse = mean_squared_error(test_y, pred)
r2 = r2_score(test_y, pred)
print("MSE is %f" %(mse))
print("R2 score is %f" %(r2))

MSE is 0.914977
R2 score is 0.581184


## Submission

In [521]:
df = pd.read_csv('..\\submission_clean.csv')

In [522]:
df.head(5)

,lag_public_perc_che_2,lag_public_perc_che_1,lag_price_unit_2,lag_price_unit_1,lag_price_month_2,lag_price_month_1,lag_insurance_perc_che_2,lag_insurance_perc_che_1,lag_che_perc_gdp_2,lag_che_perc_gdp_1,...,ind_age,che_perc_gdp,launch_date,date,insurance_perc_che,population,price_month,price_unit,public_perc_che,target
0,1.910448,1.910448,1.012037,1.012037,1.007091,1.007091,1.000000,1.000000,1.46792,1.467920,...,6,1.467920,2022-02-01,2023-01-01,1.000000,1.004739,1.007091,1.012037,1.910448,NaN
1,1.910448,1.910448,1.012037,1.012037,1.007091,1.007091,1.000000,1.000000,1.46792,1.467920,...,0,1.467920,2022-09-01,2023-01-01,1.000000,1.004739,1.157663,1.121625,1.910448,NaN
2,1.910448,1.910448,1.012037,1.121625,1.007091,1.157663,1.000000,1.000000,1.46792,1.467920,...,1,1.108030,2022-05-01,2023-01-01,1.226667,3.243906,1.157663,1.005001,1.567164,NaN
3,1.910448,1.567164,1.121625,1.005001,1.157663,1.157663,1.000000,1.226667,1.46792,1.108030,...,0,1.515375,2022-12-01,2023-01-01,1.785253,1.042681,1.157663,1.271214,1.786265,NaN
4,1.567164,1.786265,1.005001,1.271214,1.157663,1.157663,1.226667,1.785253,1.10803,1.515375,...,33,1.127864,2022-12-01,2023-01-01,1.076762,1.881992,1.157663,2.073953,1.223881,NaN


In [523]:
submit_x = df.iloc[:,:-1]

In [524]:
submit_x['date'] = pd.to_datetime(submit_x['date'])
submit_x['launch_date'] = pd.to_datetime(submit_x['launch_date'])
submit_x['age'] = (submit_x['date'].dt.year - submit_x['launch_date'].dt.year) * 12 + (submit_x['date'].dt.month - submit_x['launch_date'].dt.month)


In [525]:
submit_x.drop(['launch_date', 'date'], axis=1, inplace=True)

In [526]:
cols = submit_x.columns.tolist()
cols = cols[-1:] + cols[:-1]
submit_x = submit_x[cols]

In [527]:
submit_x

,age,lag_public_perc_che_2,lag_public_perc_che_1,lag_price_unit_2,lag_price_unit_1,lag_price_month_2,lag_price_month_1,lag_insurance_perc_che_2,lag_insurance_perc_che_1,lag_che_perc_gdp_2,...,country_encode,brand_encode,area_encode,ind_age,che_perc_gdp,insurance_perc_che,population,price_month,price_unit,public_perc_che
0,11,1.910448,1.910448,1.012037,1.012037,1.007091,1.007091,1.000000,1.000000,1.467920,...,0.185328,0.956023,0.877121,6,1.467920,1.000000,1.004739,1.007091,1.012037,1.910448
1,4,1.910448,1.910448,1.012037,1.012037,1.007091,1.007091,1.000000,1.000000,1.467920,...,0.185328,0.766304,0.319773,0,1.467920,1.000000,1.004739,1.157663,1.121625,1.910448
2,8,1.910448,1.910448,1.012037,1.121625,1.007091,1.157663,1.000000,1.000000,1.467920,...,0.742480,0.147700,0.742347,1,1.108030,1.226667,3.243906,1.157663,1.005001,1.567164
3,1,1.910448,1.567164,1.121625,1.005001,1.157663,1.157663,1.000000,1.226667,1.467920,...,0.209202,0.728905,0.319773,0,1.515375,1.785253,1.042681,1.157663,1.271214,1.786265
4,1,1.567164,1.786265,1.005001,1.271214,1.157663,1.157663,1.226667,1.785253,1.108030,...,0.978624,0.271978,0.408879,33,1.127864,1.076762,1.881992,1.157663,2.073953,1.223881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764,23,1.801799,1.704341,1.367984,1.017767,1.257193,1.157663,1.815246,1.330000,1.493446,...,0.825853,0.508115,0.877121,7,2.011926,1.573333,1.035514,1.005218,1.020516,1.283582
1765,15,1.704341,1.283582,1.017767,1.020516,1.157663,1.005218,1.330000,1.573333,1.254165,...,0.598843,0.063196,0.408879,0,1.915414,1.000000,1.179866,1.248093,1.337647,1.837175
1766,1,1.283582,1.837175,1.020516,1.337647,1.005218,1.248093,1.573333,1.000000,2.011926,...,0.004719,0.667938,0.408879,12,1.493446,1.815246,1.275586,1.196712,1.287604,1.801799
1767,1,1.837175,1.801799,1.337647,1.287604,1.248093,1.196712,1.000000,1.815246,1.915414,...,0.004719,0.667938,0.642469,12,1.493446,1.815246,1.275586,1.091414,1.604256,1.801799


In [528]:
pred = best_model.predict(submit_x)

In [529]:
origin = pd.read_csv('..\\Data Files\\submission_data.csv')

In [530]:
origin['prediction'] = pred

In [531]:
export = origin[['date', 'cluster_nl', 'prediction']]

In [532]:
export.to_csv("..\\Data Files\\submission_1.csv", index=False)